# Capstone project on explore and cluster the neighberhoods in Toronto

### 1: Data wranglering with BeautifulSoup

In [1]:
!pip install bs4
#!pip install requests

In [2]:
from bs4 import BeautifulSoup 
import requests 

import pandas as pd

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text

In [5]:
soup = BeautifulSoup(data,"html5lib")

### Tips for  Webscraping Updated Table in Week3 Peer Graded Assignment

 **After retreiving the URL and creating a Beautiful soup object** 

 **Firstly create a list**  

 **Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**

**As postal code contains upto 3 characters extract that using tablerow.p.text**

 **Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.

 **Append to the list**  

 **Create a dataframe with list**

In [6]:
## Sample code
table_contents=[]
table=soup.find('table')
len(table)

#pd.read_html(str(table), flavor='bs4')


2

In [7]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
#'''
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#'''
#df= df.groupby(['PostalCode'], as_index = False).agg({'Borough': ' '.join,'Neighborhood': ' '.join})


In [8]:
#df=df.sort_values(by=['PostalCode'], ascending=True)
#df.head(10)
#df.tail(15)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
df.shape

(103, 3)

## 2. Get the latitude and the longitude coordinates of each neighborhood using the Geocoder package

In [10]:
'''
%pip install geocoder
import geocoder # import geocoder
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
'''

"\n%pip install geocoder\nimport geocoder # import geocoder\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\n"

In [11]:
#read in CSV file downloaded to local 
df_lat_lng = pd.read_csv('Geospatial_Coordinates.csv')
df_latlng = df_lat_lng.rename(columns={"Postal Code": "PostalCode"})
#df_latlng

In [12]:
#join dataframe 
#df_geo = df.set_index('PostalCode').join(lat_lng.set_index('Postal Code'))
df_geo = df.join(df_latlng.set_index('PostalCode'), on='PostalCode')
df_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [13]:
#check Lat, Lng for PostalCode 'M5G'
df_geo.loc[df_geo['PostalCode'] == 'M5G', ['Latitude','Longitude']]

,Latitude,Longitude
24,43.657952,-79.387383


In [14]:
df_geo.shape

(103, 5)

## 3. Explore and cluster the neighborhoods in Toronto

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
%pip install folium
import folium # map rendering library

print('Libraries imported.')

Note: you may need to restart the kernel to use updated packages.
Libraries imported.


In [16]:
df_Toronto_neighborhood = df_geo
# create map of Toronto using latitude and longitude values
latitude = 43.654260
longitude = -79.360636
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto_neighborhood['Latitude'], df_Toronto_neighborhood['Longitude'], df_Toronto_neighborhood['Borough'], df_Toronto_neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='darkblue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.1 segment and cluster only the neighborhoods in Borough having "Toronto"

In [17]:
#Slice the original dataframe and create a new dataframe of the toronto_data.
toronto_data = df_Toronto_neighborhood[df_Toronto_neighborhood['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### 3.1.1 Visulaize only neighborhoods within "*Toronto" Borough

In [18]:
# create map of Toronto using latitude and longitude values
latitude = 43.654260
longitude = -79.360636
map_toronto_B = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='skyblue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_B)  
    
map_toronto_B

#### 3.1.2 Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = '1JNUAFGTJW3DQDD5C5DRLICP3FHJEPPDDO1I1XQPYJZB0CQ5' # your Foursquare ID
CLIENT_SECRET = 'MMOEN4INZO1WTWS3HUS13IQ4S5YNO5G4UM404J4WZXYWAEKE' # your Foursquare Secret
VERSION = '20210404' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1JNUAFGTJW3DQDD5C5DRLICP3FHJEPPDDO1I1XQPYJZB0CQ5
CLIENT_SECRET:MMOEN4INZO1WTWS3HUS13IQ4S5YNO5G4UM404J4WZXYWAEKE


#### 3.1.3 explore the last neighborhood

In [20]:
last_postalcode, last_borough, last_neighboehood, last_latitude, last_longitude = toronto_data.iloc[-1]
#toronto_data.tail(1)
#last_postalcode, last_borough, last_neighboehood, last_latitude, last_longitude 
print('Latitude and longitude values of {} are {}, {}.'.format(last_neighboehood, 
                                                               last_latitude, 
                                                               last_longitude))

Latitude and longitude values of Enclave of M4L are 43.6627439, -79.321558.


##### Now, let's get the top 100 venues that are inEnclave of M4L within a radius of 500 meters.

###### First, let's create the GET request URL. Name your URL url.

In [21]:
LIMIT=100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id=1JNUAFGTJW3DQDD5C5DRLICP3FHJEPPDDO1I1XQPYJZB0CQ5&client_secret=MMOEN4INZO1WTWS3HUS13IQ4S5YNO5G4UM404J4WZXYWAEKE&v=20210404&ll=43.6627439, -79.321558&radius=500&limit=100'.format(CLIENT_ID, CLIENT_SECRET, VERSION,last_latitude, last_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=1JNUAFGTJW3DQDD5C5DRLICP3FHJEPPDDO1I1XQPYJZB0CQ5&client_secret=MMOEN4INZO1WTWS3HUS13IQ4S5YNO5G4UM404J4WZXYWAEKE&v=20210404&ll=43.6627439, -79.321558&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
#results

###### Use the get_category_type function from the Foursquare lab

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

###### Now we are ready to clean the json and structure it into a pandas dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
#venues[0]   
nearby_venues = json_normalize(venues) # flatten JSON
#nearby_venues.head()
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
#nearby_venues.head()
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-24-d5a1534c7854>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rorschach Brewing Co.,Brewery,43.663483,-79.319824
1,Leslieville Farmers Market,Farmers Market,43.664901,-79.319784
2,The Sidekick,Comic Shop,43.664484,-79.325162
3,Chino Locos,Burrito Place,43.664653,-79.325584
4,Queen Margherita Pizza,Pizza Place,43.664685,-79.324164


Number of venues returned by Foursquare:

In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


#### 3.1.4 explore the all neighborhoods

create a function to repeat the same process to all the neighborhoods used from lab

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=1JNUAFGTJW3DQDD5C5DRLICP3FHJEPPDDO1I1XQPYJZB0CQ5&client_secret=MMOEN4INZO1WTWS3HUS13IQ4S5YNO5G4UM404J4WZXYWAEKE&v=20210403&ll=40.87655077879964, -73.91065965862981&radius=500&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_.

In [27]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

###### check the size of this new dataframe

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(897, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Arturo's,40.874412,-73.910271,Pizza Place
1,"Regent Park, Harbourfront",43.65426,-79.360636,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,"Regent Park, Harbourfront",43.65426,-79.360636,Tibbett Diner,40.880404,-73.908937,Diner
3,"Regent Park, Harbourfront",43.65426,-79.360636,Dunkin',40.877136,-73.906666,Donut Shop
4,"Regent Park, Harbourfront",43.65426,-79.360636,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym


###### check venues by neighborhoods

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,23,23,23,23,23,23
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",23,23,23,23,23,23
Central Bay Street,23,23,23,23,23,23
Christie,23,23,23,23,23,23
Church and Wellesley,23,23,23,23,23,23
"Commerce Court, Victoria Hotel",23,23,23,23,23,23
Davisville,23,23,23,23,23,23
Davisville North,23,23,23,23,23,23


###### check out unique catagories:

In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 20 uniques categories.


### 3.1.4.A Analyze neighborhoods:

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Gym,Ice Cream Shop,Kids Store,Pharmacy,Pizza Place,Sandwich Place,Seafood Restaurant,Steakhouse,Supplement Shop,Tennis Stadium,Video Game Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,"Regent Park, Harbourfront",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_onehot.shape

(897, 21)

###### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Gym,Ice Cream Shop,Kids Store,Pharmacy,Pizza Place,Sandwich Place,Seafood Restaurant,Steakhouse,Supplement Shop,Tennis Stadium,Video Game Store,Yoga Studio
0,Berczy Park,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
1,"Brockton, Parkdale Village, Exhibition Place",0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
3,Central Bay Street,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
4,Christie,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
5,Church and Wellesley,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
6,"Commerce Court, Victoria Hotel",0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
7,Davisville,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
8,Davisville North,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
9,"Dufferin, Dovercourt Village",0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478


In [34]:
toronto_grouped.shape

(39, 21)

###### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0  American Restaurant  0.09
1       Sandwich Place  0.09
2                  Gym  0.09
3             Pharmacy  0.04
4     Video Game Store  0.04


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0  American Restaurant  0.09
1       Sandwich Place  0.09
2                  Gym  0.09
3             Pharmacy  0.04
4     Video Game Store  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0  American Restaurant  0.09
1       Sandwich Place  0.09
2                  Gym  0.09
3             Pharmacy  0.04
4     Video Game Store  0.04


----Central Bay Street----
                 venue  freq
0  American Restaurant  0.09
1       Sandwich Place  0.09
2                  Gym  0.09
3             Pharmacy  0.04
4     Video Game Store  0.04


----Christie----
                 venue  freq
0  American Restaurant  0.09
1     

### Let's put that into a pandas dataframe

###### First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

###### let's create the new dataframe and display the top 12 venues for each neighborhood.

In [37]:
num_top_venues = 12

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,Berczy Park,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
1,"Brockton, Parkdale Village, Exhibition Place",American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
3,Central Bay Street,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
4,Christie,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store


# 4. Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 3 clusters.

In [38]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

<ipython-input-38-084c0fff10c8>:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

 Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



In [40]:
toronto_merged.tail() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
34,M5W,Downtown Toronto Stn A,Enclave of M5E,43.646435,-79.374846,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
35,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
36,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
37,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
38,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store


## Visuzlize the cluster

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clustersster

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

###### Cluster 1

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,Downtown Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
1,Downtown Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
2,Downtown Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
3,East Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
4,Downtown Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
5,Downtown Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
6,Downtown Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
7,Downtown Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
8,West Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store
9,East York/East Toronto,0,American Restaurant,Gym,Sandwich Place,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Yoga Studio,Video Game Store


###### Cluster 2

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue


###### Cluster 3

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
